<a id='home'></a>
### purpose

fit trained models from gradient forests to the climate of a transplant location (ie the populations in the simulation)


### outline

1. [read in population data](#popdata)
1. [create common garden infiles for fitting script](#cgdata)
    - create a file for each common garden - give each individual or pool the same climate data for that garden
    - use this to fit trained gradient forest as the 'future' climate of each individual/population
1. [create fitting script](#script)
    - create an R script that will use trained Gradient Forest models and fit to each of the common gardens
1. [fit to common gardens](#fit)
    - predict offset for each individual/population to the climate of its own or another population (common garden)

In [1]:
from pythonimports import *
import xarray
import rioxarray

DIR = '/work/lotterhos/MVP-Offsets/practice_slim/'
slimdir = op.join(DIR, 'mypractice')
garden_dir = makedir(op.join(DIR, 'fitting/gradient_forests/garden_files'))
fitting_dir = makedir(op.join(op.dirname(garden_dir), 'fitting_outfiles'))
training_outdir = op.join(DIR, 'training/gradient_forests/training_outfiles')

lview,dview = get_client(targets=range(20))

latest_commit()
session_info.show()

20 20
##################################################################
Current commit of pythonimports:
commit 357f2a9069d9ca25062146953c9bf88b70e863c0  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu Feb 3 10:42:21 2022 -0500
Today:	February 14, 2022 - 17:08:50
python version: 3.8.5
##################################################################



<a id='popdata'></a>
# read in population data

[top](#home)

In [2]:
seed = '1231094'

##### fitness data

In [3]:
# load fitness matrix

# an n_deme x n_deme table that indicates the mean fitness of individuals 
    # from the source deme (in columns) in the transplant deme (in rows) 
    
fitness = pd.read_table(op.join(slimdir, f'{seed}_fitnessmat.txt'),
                        delim_whitespace=True,
                        header=None)

# set column names for popID
fitness.columns = range(1, 101, 1)
fitness.index = range(1, 101, 1)

fitness.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0.960262,0.97382,0.971241,0.965472,0.976714,0.970917,0.974427,0.953202,0.954027,0.975391,0.840556,0.85031,0.832841,0.839607,0.851942,0.857426,0.892363,0.846671,0.860698,0.811939,0.661158,0.647856,0.652199,0.66758,0.645116,0.640166,0.65026,0.62293,0.584976,0.586847,0.391638,0.432267,0.404119,0.407285,0.420622,0.388318,0.410114,0.394573,0.42217,0.385656,0.206587,0.233211,0.208592,0.216491,0.199549,0.226061,0.212617,0.208645,0.20138,0.238784,0.086771,0.091085,0.090922,0.093716,0.090747,0.08962,0.096573,0.092517,0.090039,0.098542,0.027133,0.037054,0.033264,0.033311,0.031134,0.035581,0.034004,0.036603,0.038191,0.034282,0.011827,0.011865,0.00902,0.010669,0.010409,0.010367,0.010115,0.008569,0.009158,0.012367,0.003637,0.002965,0.002782,0.00337,0.002928,0.002968,0.001892,0.00305,0.0033,0.002622,0.000337,0.000261,0.000369,0.000329,0.000424,0.000411,0.000632,0.000215,0.000317,0.000258
2,0.960262,0.97382,0.971241,0.965472,0.976714,0.970917,0.974427,0.953202,0.954027,0.975391,0.840556,0.85031,0.832841,0.839607,0.851942,0.857426,0.892363,0.846671,0.860698,0.811939,0.661158,0.647856,0.652199,0.66758,0.645116,0.640166,0.65026,0.62293,0.584976,0.586847,0.391638,0.432267,0.404119,0.407285,0.420622,0.388318,0.410114,0.394573,0.42217,0.385656,0.206587,0.233211,0.208592,0.216491,0.199549,0.226061,0.212617,0.208645,0.20138,0.238784,0.086771,0.091085,0.090922,0.093716,0.090747,0.08962,0.096573,0.092517,0.090039,0.098542,0.027133,0.037054,0.033264,0.033311,0.031134,0.035581,0.034004,0.036603,0.038191,0.034282,0.011827,0.011865,0.00902,0.010669,0.010409,0.010367,0.010115,0.008569,0.009158,0.012367,0.003637,0.002965,0.002782,0.00337,0.002928,0.002968,0.001892,0.00305,0.0033,0.002622,0.000337,0.000261,0.000369,0.000329,0.000424,0.000411,0.000632,0.000215,0.000317,0.000258
3,0.960262,0.97382,0.971241,0.965472,0.976714,0.970917,0.974427,0.953202,0.954027,0.975391,0.840556,0.85031,0.832841,0.839607,0.851942,0.857426,0.892363,0.846671,0.860698,0.811939,0.661158,0.647856,0.652199,0.66758,0.645116,0.640166,0.65026,0.62293,0.584976,0.586847,0.391638,0.432267,0.404119,0.407285,0.420622,0.388318,0.410114,0.394573,0.42217,0.385656,0.206587,0.233211,0.208592,0.216491,0.199549,0.226061,0.212617,0.208645,0.20138,0.238784,0.086771,0.091085,0.090922,0.093716,0.090747,0.08962,0.096573,0.092517,0.090039,0.098542,0.027133,0.037054,0.033264,0.033311,0.031134,0.035581,0.034004,0.036603,0.038191,0.034282,0.011827,0.011865,0.00902,0.010669,0.010409,0.010367,0.010115,0.008569,0.009158,0.012367,0.003637,0.002965,0.002782,0.00337,0.002928,0.002968,0.001892,0.00305,0.0033,0.002622,0.000337,0.000261,0.000369,0.000329,0.000424,0.000411,0.000632,0.000215,0.000317,0.000258
4,0.960262,0.97382,0.971241,0.965472,0.976714,0.970917,0.974427,0.953202,0.954027,0.975391,0.840556,0.85031,0.832841,0.839607,0.851942,0.857426,0.892363,0.846671,0.860698,0.811939,0.661158,0.647856,0.652199,0.66758,0.645116,0.640166,0.65026,0.62293,0.584976,0.586847,0.391638,0.432267,0.404119,0.407285,0.420622,0.388318,0.410114,0.394573,0.42217,0.385656,0.206587,0.233211,0.208592,0.216491,0.199549,0.226061,0.212617,0.208645,0.20138,0.238784,0.086771,0.091085,0.090922,0.093716,0.090747,0.08962,0.096573,0.092517,0.090039,0.098542,0.027133,0.037054,0.033264,0.033311,0.031134,0.035581,0.034004,0.036603,0.038191,0.034282,0.011827,0.011865,0.00902,0.010669,0.010409,0.010367,0.010115,0.008569,0.009158,0.012367,0.003637,0.002965,0.002782,0.00337,0.002928,0.002968,0.001892,0.00305,0.0033,0.002622,0.000337,0.000261,0.000369,0.000329,0.000424,0.000411,0.000632,0.000215,0.000317,0.000258
5,0.960262,0.97382,0.971241,0.965472,0.976714,0.970917,0.974427,0.953202,0.954027,0.975391,0.840556,0.85031,0.832841,0.839607,0.851942,0.857426,0.89236

##### population location

In [4]:
# get coordinates for each population

subset = pd.read_table(op.join(slimdir, f'{seed}_Rout_ind_subset.txt'),
                       delim_whitespace=True)
subset.index = ('i' + subset['indID'].astype(str)).tolist()  # this will match to the 'causal' file
subset['sample_name'] = subset.index.tolist()

print(f'{nrow(subset) = }')

locations = subset.groupby('subpop')[['x', 'y']].apply(np.mean)
locations.columns = ['lon', 'lat']
locations.head()

nrow(subset) = 1000


,lon,lat
subpop,,
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
5,5.0,1.0


In [5]:
subset.head()

,seed,subpopID,indID,indSubpopIndex,subpop,phen_sal,phen_temp,sal_opt,temp_opt,fitness,subset,N,opt0,opt1,x,y,PC1,PC2,PC3,LFMM_U1_temp,LFMM_U1_sal,LFMM_U2_temp,LFMM_U2_sal,RDA1,RDA2,RDA_PC1,RDA_PC2,RDA_predict_tempPhen_20KSNPs,RDA_predict_salPhen_20KSNPs,sample_name
i0,1231094,1,0,0,1,0,-1.088090,-1.0,-1.0,0.984601,True,10,-1.0,-1.0,1,1,92.2154,-57.5673,-42.4980,-38.876285,72.935933,14.058057,40.216243,-2.464484,-4.413873,-2.401906,-2.096819,-5924.695082,-64.166353,i0
i1,1231094,1,1,1,1,0,-0.771927,-1.0,-1.0,0.901194,True,10,-1.0,-1.0,1,1,92.5184,-58.2218,-44.4978,-39.042619,73.119916,15.385371,40.384374,-2.472104,-3.013454,-2.412105,-2.206939,-5943.060707,-49.737004,i1
i2,1231094,1,2,2,1,0,-0.883133,-1.0,-1.0,0.973054,True,10,-1.0,-1.0,1,1,92.7162,-59.2298,-43.8789,-40.110397,73.313012,14.814410,41.455827,-2.478389,-3.995747,-2.476676,-2.159522,-5958.136915,-59.945481,i2
i3,1231094,1,3,3,1,0,-0.836002,-1.0,-1.0,0.947630,True,10,-1.0,-1.0,1,1,90.1391,-54.8804,-40.6813,-36.756808,71.370522,13.047852,38.074237,-2.409776,-2.373063,-2.273755,-2.013461,-5793.238241,-42.644448,i3
i4,1231094,1,4,4,1,0,-1.078760,-1.0,-1.0,0.987670,True,10,-1.0,-1.0,1,1,91.8586,-56.6580,-42.9090,-38.019264,72.297632,14.507998,39.346027,-2.443488,-4.116506,-2.350035,-2.134117,-5874.228034,-60.932578,i4


In [6]:
envdir = op.join(DIR, 'training/gradient_forests/training_files')
files = fs(envdir, 'envfile')

envfiles = {}
for f in files:
    ind_or_pooled = op.basename(f).split("_")[-1].rstrip(".txt")
    envfiles[ind_or_pooled] = f
    
envfiles

{'ind': '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_files/1231094_envfile_GFready_ind.txt',
 'pooled': '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_files/1231094_envfile_GFready_pooled.txt'}

In [7]:
envdfs = {}
for ind_or_pooled,f in envfiles.items():
    envdfs[ind_or_pooled] = pd.read_table(f, index_col=0)
    print(ColorText(ind_or_pooled).bold())
    display(envdfs[ind_or_pooled].head())

ind


,sal_opt,temp_opt
i0,-1.0,-1.0
i1,-1.0,-1.0
i2,-1.0,-1.0
i3,-1.0,-1.0
i4,-1.0,-1.0


pooled


,sal_opt,temp_opt
1,-1.000000,-1.0
2,-0.777778,-1.0
3,-0.555556,-1.0
4,-0.333333,-1.0
5,-0.111111,-1.0


<a id='cgdata'></a>
# create common garden infiles to fitting script

[top](#home)

In [8]:
# the population environments
envdfs['pooled']

,sal_opt,temp_opt
1,-1.000000,-1.0
2,-0.777778,-1.0
3,-0.555556,-1.0
4,-0.333333,-1.0
5,-0.111111,-1.0
...,...,...
96,0.111111,1.0
97,0.333333,1.0
98,0.555556,1.0
99,0.777778,1.0


In [9]:
# create a file that has the same climate for each individual/pool for each transplant garden 
    # ie the future climate
garden_files = defaultdict(list)
for ind_or_pooled,f in envfiles.items():  # for each way of training gradient forests
    df = envdfs[ind_or_pooled].copy()
    for subpopID in pbar(envdfs['pooled'].index, desc=ind_or_pooled):  # for each garden's climate
        # set all individuals/pools to the same value
        df.loc[:] = envdfs['pooled'].loc[subpopID, df.columns].tolist()
        # save
        garden_file = op.join(garden_dir, op.basename(f).replace(".txt", f"_{subpopID}.txt"))
        df.to_csv(garden_file, sep='\t', index=True)
        # record
        assert garden_file not in garden_files[ind_or_pooled]
        garden_files[ind_or_pooled].append(garden_file)

df

pooled: 100%|███████████████| 100/100 [00:00<00:00, 111.12it/s]


,sal_opt,temp_opt
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,1.0
...,...,...
96,1.0,1.0
97,1.0,1.0
98,1.0,1.0
99,1.0,1.0


In [10]:
# example from loop above
garden_file

'/work/lotterhos/MVP-Offsets/practice_slim/fitting/gradient_forests/garden_files/1231094_envfile_GFready_pooled_100.txt'

<a id='script'></a>
# create fitting script

[top](#home)

In [11]:
DIR

'/work/lotterhos/MVP-Offsets/practice_slim/'

In [12]:
text = '''
#----------------------------------------------------------------------------------------------------------#
# Given a trained gradient forest, fit model to input climate data, `garden_data`.
#
# Usage
# -----
# Rscript gradient_fitting_script.R gfOut_trainingfile range_file predfile maskfile basename save_dir
#
# Parameters
# ----------
# gfOut_trainingfile - file path to trained gradient forest object, saved as .RDS
# garden_file - tab delimited .txt file path with columns for `lat`, `lon`, and each env used in training
#    - data can be current, future, or uniform (eg for common garden) climate
# predfile - path to RDS of object saved from interpolating gradient forest model across landscape
#    for the same time period as the data that went into training (created in gradient_training.R)
# basename - the basename prefix wanted for saving files in `save_dir`
# save_dir - the directory where files should be saved
#
# Notes
# -----
# - This file is based on gradient_fitting_script.R from github.com/brandonlind/offset_validation
# - Differences include (but may not be limited to):
#     - output not written to .nc file
#     - garden_file input assumed to have row names
#     - garden_file assumes only environmental columns (no extras)
#----------------------------------------------------------------------------------------------------------#

library(gradientForest)


calc_euclid <- function(proj, curr){
    #-------------------------------------------------------------------#
    # CALCULATE EUCLIDEAN DISTANCE BETWEEN CURRENT AND PROJECTED OFFSET
    #-------------------------------------------------------------------#
    
    df <- data.frame(matrix(ncol=ncol(proj), nrow=nrow(proj)))
    stopifnot(all(colnames(proj) == colnames(curr)))
    for (i in 1:ncol(proj)){
        # square each difference between the column data
        df[,i] <- (proj[,i] - curr[,i])**2
    }
    # sum by rows then take the square root of each
    return (sqrt(apply(df, 1, sum)))
}


# COMMAND LINE ARGS
args = commandArgs(trailingOnly=TRUE)
gfOut_trainingfile <- args[1]
garden_file        <- args[2]
predfile           <- args[3]
basename           <- args[4]
save_dir           <- args[5]


# SET UP NAMESPACE
cat(sprintf('\\nLoading data ...'))
gfOut <- readRDS(gfOut_trainingfile)  # output from gradient_training.R
garden_data <- read.csv(garden_file, sep='\\t', row.names=1)
predOut <- readRDS(predfile)  # output from gradient_training.R


# PREDICT OFFSET FOR COMMON GARDEN
cat(sprintf('\\nProjecting to climate of common garden ...'))
projOut <- predict(gfOut, garden_data)


# CALC OFFSET
cat(sprintf('\\nCalculating offset ...'))
offset <- calc_euclid(projOut, predOut)
offset_df <- data.frame(offset)
rownames(offset_df) <- rownames(garden_data)


# SAVE PROJECTED OFFSET OBJECT
cat(sprintf('\\nSaving offset object ...'))
outfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_fitted.RDS', basename)
)
saveRDS(projOut, outfile)
print(outfile)


# SAVE OFFSET DATA
cat(sprintf('\\nSaving offset data ...'))
offsetfile <- paste0(
    save_dir,
    sprintf('/%s_gradient_forest_offset.txt', basename)
)
write.table(offset_df, offsetfile, sep='\\t')
print(offsetfile)

'''

fitting_file = '/home/b.lind/code/MVP-offsets/00_explore_sims_output/MVP_GF_fitting_script.R'
with open(fitting_file, 'w') as o:
    o.write(text)

print(text)



#----------------------------------------------------------------------------------------------------------#
# Given a trained gradient forest, fit model to input climate data, `garden_data`.
#
# Usage
# -----
# Rscript gradient_fitting_script.R gfOut_trainingfile range_file predfile maskfile basename save_dir
#
# Parameters
# ----------
# gfOut_trainingfile - file path to trained gradient forest object, saved as .RDS
# garden_file - tab delimited .txt file path with columns for `lat`, `lon`, and each env used in training
#    - data can be current, future, or uniform (eg for common garden) climate
# predfile - path to RDS of object saved from interpolating gradient forest model across landscape
#    for the same time period as the data that went into training (created in gradient_training.R)
# basename - the basename prefix wanted for saving files in `save_dir`
# save_dir - the directory where files should be saved
#
# Notes
# -----
# - This file is based on gradient_fitting_script.R

<a id='fit'></a>
# fit to common gardens

[top](#home)

In [13]:
# get files for this simulation seed output by gradient forests (samples: ind_or_pooled, loci: adaptive_or_all)
predfiles = fs(training_outdir,
               pattern=f'{seed}_',
               endswith='predOut.RDS')

predfiles

['/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_ind_adaptive_gradient_forest_predOut.RDS',
 '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_pooled_adaptive_gradient_forest_predOut.RDS',
 '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_pooled_all_gradient_forest_predOut.RDS']

In [14]:
def fit_gradient_forests(gfOut_trainingfile, garden_file, predfile, basename, save_dir):
    """Fit trained Gradient Forest model using `fitting_file` from above."""
    import subprocess
    
    output = subprocess.check_output(
        [
            Rscript,
            fitting_file,
            gfOut_trainingfile,
            garden_file,
            predfile,
            basename,
            save_dir            
        ]
    ).decode('utf-8')
    
    return output

Rscript = '/home/b.lind/anaconda3/envs/r35/bin/Rscript'
dview['Rscript'] = Rscript
dview['fitting_file'] = fitting_file

In [15]:
# set up input arguments to `fit_gradient_forests` function
args = []
basenames = []
for predfile in predfiles:
    trainingfile = predfile.replace("_predOut.", "_training.")
    assert op.exists(trainingfile)
    
    # was this predfile from training with individuals or pools? with adaptive loci or all?
    ind_or_pooled, adaptive_or_all = op.basename(predfile)\
                                        .split("GF_training_")[1]\
                                        .split("_gradient_forest")[0]\
                                        .split('_')
    
    for garden_file in garden_files[ind_or_pooled]:
        garden_ID = op.basename(garden_file).split("_")[-1].rstrip(".txt")
        
        basename = f'{seed}_{ind_or_pooled}_{adaptive_or_all}_{garden_ID}'
        assert basename not in basenames
        basenames.append(basename)
        
        _args = (trainingfile,
                 garden_file,
                 predfile,
                 basename,
                 fitting_dir)

        args.append(_args)
        
# test function
output = fit_gradient_forests(*_args)
print(output)

Loading required package: extendedForest
extendedForest 1.6.1
Type rfNews() to see new features/changes/bug fixes.



Loading data ...
Projecting to climate of common garden ...
Calculating offset ...
Saving offset object ...[1] "/work/lotterhos/MVP-Offsets/practice_slim/fitting/gradient_forests/fitting_outfiles/1231094_pooled_all_100_gradient_forest_fitted.RDS"

Saving offset data ...[1] "/work/lotterhos/MVP-Offsets/practice_slim/fitting/gradient_forests/fitting_outfiles/1231094_pooled_all_100_gradient_forest_offset.txt"



In [16]:
_args

('/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_pooled_all_gradient_forest_training.RDS',
 '/work/lotterhos/MVP-Offsets/practice_slim/fitting/gradient_forests/garden_files/1231094_envfile_GFready_pooled_100.txt',
 '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_pooled_all_gradient_forest_predOut.RDS',
 '1231094_pooled_all_100',
 '/work/lotterhos/MVP-Offsets/practice_slim/fitting/gradient_forests/fitting_outfiles')

In [17]:
# look at output from testing the fitting script
pd.read_table("/work/lotterhos/MVP-Offsets/practice_slim/fitting/gradient_forests/fitting_outfiles/1231094_pooled_all_100_gradient_forest_offset.txt")

,offset
1,0.716106
2,0.716106
3,0.716105
4,0.716105
5,0.716105
...,...
96,0.000645
97,0.000509
98,0.000360
99,0.000221


In [19]:
# fit everything in parallel
lview,dview = get_client(targets=range(7))  # only use 7 engines so I don't max out mem

jobs = []
for _args in args:
    jobs.append(
        lview.apply_async(fit_gradient_forests, *_args)
    )

watch_async(jobs)

7 7

Watching 300 jobs ...


100%|███████████████| 300/300 [28:14<00:00,  5.65s/it]  
